# TIR Collection

## Loading relevant libraries

`tidyverse` is an R library that contains core packages used to read, analyze, and plot data

In [1]:
library(tidyverse)

Registered S3 methods overwritten by 'ggplot2':
  method         from 
  [.quosures     rlang
  c.quosures     rlang
  print.quosures rlang
Registered S3 method overwritten by 'rvest':
  method            from
  read_xml.response xml2
-- Attaching packages --------------------------------------- tidyverse 1.2.1 --
v ggplot2 3.1.1       v purrr   0.3.2  
v tibble  2.1.1       v dplyr   0.8.0.1
v tidyr   0.8.3       v stringr 1.4.0  
v readr   1.3.1       v forcats 0.4.0  
-- Conflicts ------------------------------------------ tidyverse_conflicts() --
x dplyr::filter() masks stats::filter()
x dplyr::lag()    masks stats::lag()


## Dexcom
Given that the exported .csv files for Dexcom and FreeStyle Libre are formatted differently, reading and wrangling must also be carried out differently. Following this, the formula for calculating TIR is the same for both CGMS.

### Defining Function

In [4]:
dexcom <- function(file) {
    suppressWarnings({
        read <- read_csv(file)
        
        # renaming columns for ease of modifying
        names(read)[8] = 'Glucose_Value'
        names(read)[2] = 'Timestamp'
        names(read)[14] = 'Transmitter_ID'
        names(read)[5] = 'Patient_Info'
        
        # creating string based on user name and birthdate for id
        info <- filter(read, Patient_Info != 'NA')
        info_string <- info %>% pull(Patient_Info)
        id_string <- paste(info_string, collapse = '')
        id_string
        
        # removing top 11 rows that contain user's name and alert types
        rows <- filter(read, Transmitter_ID != 'NA') %>%
                filter(Glucose_Value != 'NA')
        
        # selecting only timestamp and glucose value columns
        cols <- select(rows, Timestamp, Glucose_Value)
        
        # pulling vectors from columns
        glucose_vector_str <- cols %>% pull(Glucose_Value)
        glucose_vector <- as.double(glucose_vector_str) # as decimals
        
        time_vector <- cols %>% pull(Timestamp)
        minutes <- minutes(time_vector) # pulling the number of readings (in minutes)
        
        
        # performing calculations
        Mean <- mean(glucose_vector, na.rm=TRUE) # average glucose
        GMI <- 12.71 + 4.70587 * Mean # glucose management indicator (mmol/mol)
        SD <- sd(glucose_vector, na.rm=TRUE) # standard deviation
        CV <- SD / Mean # coefficient of variation
        
        # time in range
        in_range <- sum(glucose_vector >= 3.9 & glucose_vector <= 10.0, na.rm=TRUE)
        TIR <- round(in_range * 1000 / length(minutes)) / 10
        
        # very low
        very_low <- sum(glucose_vector < 3.0, na.rm=TRUE)
        TBR_VL <- round(very_low * 1000 / length(minutes)) / 10
        
        # low
        low <- sum(glucose_vector >= 3.0 & glucose_vector <= 3.8, na.rm=TRUE)
        TBR_L <- round(low * 1000 / length(minutes)) / 10
        
        # high
        high <- sum(glucose_vector >= 10.1 & glucose_vector <= 13.9, na.rm=TRUE)
        TAR_H <- round(high * 1000 / length(minutes)) / 10
        
        # very high
        very_high <- sum(glucose_vector > 13.9, na.rm=TRUE)
        TAR_VH <- round(very_high * 1000 / length(minutes)) / 10
        
        # data frame
        df <- data.frame(id_string, TIR, TBR_VL, TBR_L, TAR_H, TAR_VH, GMI, SD, CV)
        
    })
}

### Applying Function
Calling the `dexcom` function on all .csv files in the Dexcom folder to create a data frame with all data from participants who use Dexcom. 

In the final data frame and .csv file (adding both Dexcom and Freestyle Libre using participants to a single data frame), the participants' `id_string`, i.e. their name and birthday, will not be visible and they will be assigned to a unique ID.

In [5]:
dex_data <- list.files(path = 'data/Dexcom',    
                       pattern = "*.csv", full.names = TRUE) %>% 
  lapply(dexcom) %>%                                           
  bind_rows
  
dex_data

Parsed with column specification:
cols(
  Index = col_double(),
  `Timestamp (YYYY-MM-DDThh:mm:ss)` = col_datetime(format = ""),
  `Event Type` = col_character(),
  `Event Subtype` = col_character(),
  `Patient Info` = col_character(),
  `Device Info` = col_character(),
  `Source Device ID` = col_character(),
  `Glucose Value (mmol/L)` = col_character(),
  `Insulin Value (u)` = col_logical(),
  `Carb Value (grams)` = col_logical(),
  `Duration (hh:mm:ss)` = col_time(format = ""),
  `Glucose Rate of Change (mmol/L/min)` = col_double(),
  `Transmitter Time (Long Integer)` = col_double(),
  `Transmitter ID` = col_character()
)
Parsed with column specification:
cols(
  Index = col_double(),
  `Timestamp (YYYY-MM-DDThh:mm:ss)` = col_datetime(format = ""),
  `Event Type` = col_character(),
  `Event Subtype` = col_character(),
  `Patient Info` = col_character(),
  `Device Info` = col_character(),
  `Source Device ID` = col_character(),
  `Glucose Value (mmol/L)` = col_character(),
  `Insulin 

id_string,TIR,TBR_VL,TBR_L,TAR_H,TAR_VH,GMI,SD,CV
JonDoe1/3/1959,49.4,0.3,1.1,31.6,17.3,61.16917,3.764559,0.3655763
JonDoe1/3/1959,49.4,0.3,1.1,31.6,17.3,61.17087,3.764705,0.3655777
LisaDoe1/3/1960,49.4,0.3,1.1,31.6,17.3,61.17087,3.764705,0.3655777
JamesMadison1999-08-30,71.1,0.0,0.3,24.0,4.5,54.74512,2.545870,0.2850125


## Freestyle Libre

### Defining Function

In [6]:
libre <- function(file) {
    suppressWarnings({
        
        # initial reading to create id
        read_id <- read_csv(file)
        names(read_id)[1] = 'Patient_report'
        names(read_id)[2] = 'Generated_on'
        
        # creating string based on user name and birthdate for id
        info <- filter(read_id, Patient_report != 'FreeStyle Libre 2') %>%
                filter(Patient_report != 'FreeStyle LibreLink') %>%
                filter(Patient_report != 'Device') %>%
                select(Patient_report, Generated_on)
        patient_string <- info %>% pull(Patient_report)
        date_string <- info %>% pull(Generated_on)
        id_string <- paste(patient_string, date_string, collapse = '')
        
        read <- read_csv(file, skip=2)
        
        # renaming columns for ease of modifying
        names(read)[3]='Timestamp'
        names(read)[5]='Glucose_Value'
        
        # removing NA values in Glucose_Value column
        rows <- filter(read, Glucose_Value != 'NA')
        
        # selecting only timestamp and glucose value columns
        cols <- select(rows, Timestamp, Glucose_Value)
        
        # pulling vectors from columns
        glucose_vector <- cols %>% pull(Glucose_Value)
        
        time_vector <- cols %>% pull(Timestamp)
        time_dttm <- ymd_hms(time_vector)
        minutes <- minutes(time_dttm) # pulling the number of readings (in minutes)
        
        # performing calculations
        Mean <- mean(glucose_vector, na.rm=TRUE) # average glucose
        GMI <- 12.71 + 4.70587 * Mean # glucose management indicator (mmol/mol)
        SD <- sd(glucose_vector, na.rm=TRUE) # standard deviation
        CV <- SD / Mean # coefficient of variation
        
        # time in range
        in_range <- sum(glucose_vector >= 3.9 & glucose_vector <= 10.0, na.rm=TRUE)
        TIR <- round(in_range * 1000 / length(minutes)) / 10
        
        # very low
        very_low <- sum(glucose_vector < 3.0, na.rm=TRUE)
        TBR_VL <- round(very_low * 1000 / length(minutes)) / 10
        
        # low
        low <- sum(glucose_vector <= 3.8 & glucose_vector >= 3.0, na.rm=TRUE)
        TBR_L <- round(low * 1000 / length(minutes)) / 10
        
        # high
        high <- sum(glucose_vector >= 10.1 & glucose_vector <= 13.9, na.rm=TRUE)
        TAR_H <- round(high * 1000 / length(minutes)) / 10
        
        # very high
        very_high <- sum(glucose_vector > 13.9, na.rm=TRUE)
        TAR_VH <- round(very_high * 1000 / length(minutes)) / 10
        
        # data frame
        df <- data.frame(id_string, TIR, TBR_VL, TBR_L, TAR_H, TAR_VH, GMI, SD, CV)
        
    })
}

### Applying Function

Calling the `libre` function on all .csv files in the Libre folder to create a data frame with all data from participants who use Freestyle Libre.

In [7]:
libre_data <- list.files(path = 'data/Libre',    
                       pattern = "*.csv", full.names = TRUE) %>% 
  lapply(libre) %>%                                           
  bind_rows                                                      

libre_data

Parsed with column specification:
cols(
  `Patient report` = col_character(),
  `Generated on` = col_character(),
  `07-08-2023 16:48 UTC` = col_character(),
  `Generated by` = col_character(),
  `Nabeel Khan` = col_character()
)
Parsed with column specification:
cols(
  Device = col_character(),
  `Serial Number` = col_character(),
  `Device Timestamp` = col_character(),
  `Record Type` = col_double(),
  `Historic Glucose mmol/L` = col_double(),
  `Scan Glucose mmol/L` = col_logical(),
  `Non-numeric Rapid-Acting Insulin` = col_logical(),
  `Rapid-Acting Insulin (units)` = col_logical(),
  `Non-numeric Food` = col_logical(),
  `Carbohydrates (grams)` = col_logical(),
  `Carbohydrates (servings)` = col_logical(),
  `Non-numeric Long-Acting Insulin` = col_logical(),
  `Long-Acting Insulin (units)` = col_logical(),
  Notes = col_logical(),
  `Strip Glucose mmol/L` = col_logical(),
  `Ketone mmol/L` = col_logical(),
  `Meal Insulin (units)` = col_logical(),
  `Correction Insulin (units)` 

id_string,TIR,TBR_VL,TBR_L,TAR_H,TAR_VH,GMI,SD,CV
Jonathan Doe 11-09-1971,98.7,0,0.5,0.8,0.0,42.25880,1.137257,0.1811168
Jon Doe 19-11-1963,56.3,0,0.0,43.4,0.3,60.04153,1.248841,0.1241643
Lisa Doe 27-11-1964,61.8,0,0.0,34.9,3.2,58.43489,1.954583,0.2011599


## Combining Dexcom and FreeStyle Libre Data

Binding the dataframes from Dexcom users and Freestyle Libre users together into a single dataframe, which is written into a .csv file that can be downloaded.

In [8]:
output_file <- 'final_df.csv'

collection <- rbind(dex_data, libre_data)

collection_id <- transform(collection, ID = as.numeric(factor(id_string)))

final_collection <- collection_id %>% select(TIR, TBR_VL, TBR_L, TAR_H, TAR_VH, GMI, SD, CV, ID) %>% arrange(ID)
final_collection
write.csv(final_collection, output_file)

TIR,TBR_VL,TBR_L,TAR_H,TAR_VH,GMI,SD,CV,ID
71.1,0.0,0.3,24.0,4.5,54.74512,2.545870,0.2850125,1
56.3,0.0,0.0,43.4,0.3,60.04153,1.248841,0.1241643,2
98.7,0.0,0.5,0.8,0.0,42.25880,1.137257,0.1811168,3
49.4,0.3,1.1,31.6,17.3,61.16917,3.764559,0.3655763,4
49.4,0.3,1.1,31.6,17.3,61.17087,3.764705,0.3655777,4
61.8,0.0,0.0,34.9,3.2,58.43489,1.954583,0.2011599,5
49.4,0.3,1.1,31.6,17.3,61.17087,3.764705,0.3655777,6
